In [6]:
import pandas as pd
import numpy as np
import math as math
import statistics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import datetime
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_decision_regions
import time

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', 1000)  #
pd.set_option('display.width', 150)

# from functions import *
#https://www.nba.com/stats/teams/boxscores-scoring

In [7]:
def convertDateColumn(df):
    locationToInsert = df.columns.get_loc('DATE')+1
    dates = pd.to_datetime(df['DATE'],format="%m/%d/%y").dt.date
    df = df.drop(columns='DATE')
    df.insert(locationToInsert,'DATE',dates)
    df = df.sort_values(by=['DATE','GAMEID']).reset_index(drop=True)
    return df
def featureEngineering(boxScoreType,df):
    if boxScoreType == 'TRADITIONAL':
        featuresToAddDict = {'OPPT':[],'PTSA':[],'2FGA':[],'2FGM':[],'2FG%':[],
                             'RESULT':[],'PTSDELTA':[],'H/A':[]}
        for i in range(len(df)):
            featuresToAddDict['OPPT'].append(df.iloc[i]['MATCH'][-3:])
            if i % 2 == 0: featuresToAddDict['PTSA'].append(df.iloc[i+1]['PTS'])
            else: featuresToAddDict['PTSA'].append(df.iloc[i-1]['PTS'])
            if df['W/L'][i] == 'L': featuresToAddDict['RESULT'].append(0)
            else: featuresToAddDict['RESULT'].append(1)
            featuresToAddDict['H/A'].append(df['MATCH'][i].count('vs'))
        df.insert(df.columns.get_loc('TEAM')+1,'OPPT',featuresToAddDict['OPPT'])
        df.insert(df.columns.get_loc('PTS')+1,'PTSA',featuresToAddDict['PTSA'])
        df.insert(df.columns.get_loc('FG%')+1,'2FGM',df['FGM'] - df['3PM'])
        df.insert(df.columns.get_loc('FG%')+1,'2FGA',df['FGA'] - df['3PA']) 
        df.insert(df.columns.get_loc('FG%')+1,'2FG%',df['2FGM']/df['2FGA'])
        df.insert(df.columns.get_loc('W/L')+1,'RESULT',featuresToAddDict['RESULT'])
        df = df.drop(columns=['W/L'])
        df.insert(df.columns.get_loc('PTSA')+1,'PTSDELTA',df['PTS']-df['PTSA'])
        df.insert(df.columns.get_loc('DATE')+1,'H/A',featuresToAddDict['H/A'])
        return df
    else:
        featuresToAddDict = {"OPPT":[],"RESULT":[],"H/A":[]}
        for i in range(len(df)):
            featuresToAddDict["OPPT"].append(df.iloc[i]['MATCH'][-3:])
            if df['W/L'][i] == 'L': featuresToAddDict["RESULT"].append(0)
            else: featuresToAddDict["RESULT"].append(1)
            featuresToAddDict["H/A"].append(df['MATCH'][i].count('vs'))
        df.insert(df.columns.get_loc('TEAM')+1,'OPPT',featuresToAddDict["OPPT"])
        df.insert(df.columns.get_loc('W/L')+1,'RESULT',featuresToAddDict["RESULT"])
        df = df.drop(columns=['W/L'])
        df.insert(df.columns.get_loc('DATE')+1,'H/A',featuresToAddDict["H/A"])
        return df
def addDaysOff(teamsDict):
    for team in list(teamsDict.keys()):
        daysOff = []
        daysOff.append(np.nan)
        for i in range(1,len(teamsDict[team]['DATE'])):
            date = teamsDict[team]['DATE'][i]
            prevDate = teamsDict[team]['DATE'][i-1] + datetime.timedelta(1)
            daysOff.append((date - prevDate).days)
        teamsDict[team]['DAYSOFF'] = daysOff
def addAdditionalFeatures(df,teamsDict):
    teams = []
    for team in list(teamsDict.keys()):
        teamDF = df[df['TEAM'] == team]
        startingIndex = len(list(df.columns))
        additionalFeatures = list(teamsDict[team].keys())[startingIndex:]
        for feature in additionalFeatures:
            teamDF = pd.concat([teamDF,
                                pd.DataFrame(teamsDict[team][feature],columns=[feature],index=teamDF.index.values)],axis=1)
        teams.append(teamDF)
    return pd.concat(teams)

def getTeamData(boxScoreType,df):
    teamsDict = {}
    features = df.columns.values
    
    if boxScoreType == 'TRADITIONAL': additionalFeatures = ['Ws','Ls','GP','W%','L%','AVGPTS','AVGPTSA']
    else: additionalFeatures = ['GP']
    for team in np.unique(df['TEAM']):
        teamsDict[team] = {}
    for team in np.unique(df['TEAM']):
        for feature in features:
            teamsDict[team][feature] = []
        for feature in additionalFeatures:
            teamsDict[team][feature] = []
    for i in range(len(df)):
        for feature in features:
            team = df.iloc[i]['TEAM']
            teamsDict[team][feature].append(df.iloc[i][feature])
        for feature in additionalFeatures:
            if feature == 'Ws': teamsDict[team][feature].append(teamsDict[team]['RESULT'].count(1))
            if feature == 'Ls': teamsDict[team][feature].append(teamsDict[team]['RESULT'].count(0))
            if feature == 'GP': teamsDict[team][feature].append(len(teamsDict[team]['RESULT']))
            if feature == 'W%': teamsDict[team][feature].append(teamsDict[team]['Ws'][-1]/teamsDict[team]['GP'][-1])
            if feature == 'L%': teamsDict[team][feature].append(teamsDict[team]['Ls'][-1]/teamsDict[team]['GP'][-1])
            if feature == 'AVGPTS': teamsDict[team][feature].append(np.mean(teamsDict[team]['PTS']))
            if feature == 'AVGPTSA': teamsDict[team][feature].append(np.mean(teamsDict[team]['PTSA']))
    return teamsDict

def addMovingAverageLx(featureStringsDict,teamDict,x):
    for team in list(teamsDict.keys()):
        for feature in featureStringsDict.keys():
            ma = []
            for _ in range(x): ma.append(np.nan)
            for maRange in range(0,len(teamsDict[team][feature])-x):
                ma.append(np.mean(teamsDict[team][feature][maRange:(x+maRange)]))
            teamsDict[team][featureStringsDict[feature].replace('x',str(x))] = ma
                
def addPercentMovingAverageLx(featureStringsDict,teamsDict,x):
    for team in list(teamsDict.keys()):
        for features in featureStringsDict.keys():
            ma = []
            for _ in range(x): ma.append(np.nan)
            for maRange in range(0,len(teamsDict[team][features[0]])-x):
                num = np.sum(teamsDict[team][features[0]][maRange:(x+maRange)])
                dem = np.sum(teamsDict[team][features[1]][maRange:(x+maRange)])
                ma.append(num/dem)
            teamsDict[team][featureStringsDict[features].replace('x',str(x))] = ma

# Traditional.csv

In [10]:
start_time = time.time()
traditionalDF = pd.read_csv('/Users/olutosinfashusi/jupyter/csv/fromNBADOTCOM/2023-24/byGame/Traditional.csv')
traditionalDF = convertDateColumn(traditionalDF)
traditionalDF = featureEngineering('TRADITIONAL',traditionalDF)
teamsDict = getTeamData('TRADITIONAL',traditionalDF)
print("%s seconds" % (time.time() - start_time))

31.74401879310608 seconds


In [9]:
# start_time = time.time()
# trainDF = pd.read_csv(
#     '/Users/olutosinfashusi/jupyter/csv/fromNBADOTCOM/2023-24/byGame/Traditional.csv'
# )
# trainDF = convertCSVDateColumn(trainDF)
# FE_trainDF = featureEngineering(trainDF)
# dict_FE_trainDF = getDataFromDataFrame(FE_trainDF)
# # getMovingAverageLx(dict_FE_trainDF,5)
# # getPercentMovingAverageLx(dict_FE_trainDF,5)
# # getMovingAverageLx(dict_FE_trainDF,4)
# # getPercentMovingAverageLx(dict_FE_trainDF,4)
# # getMovingAverageLx(dict_FE_trainDF,3)
# # getPercentMovingAverageLx(dict_FE_trainDF,3)
# # getMovingAverageLx(dict_FE_trainDF,2)
# # getPercentMovingAverageLx(dict_FE_trainDF,2)
# # getMovingAverageLx(dict_FE_trainDF,1)
# # getPercentMovingAverageLx(dict_FE_trainDF,1)
# # getDaysOff(dict_FE_trainDF)
# AF_FE_trainDF = addAdditionalFeatures(FE_trainDF, dict_FE_trainDF)
# print("%s seconds" % (time.time() - start_time))

In [22]:
NBAdf = traditionalDF.sort_values(by=['DATE', 'GAMEID', 'H/A']).reset_index(drop=True)
NBAteams = np.unique(NBAdf['TEAM'])

In [31]:
dict_ = {}
for team in NBAteams:
    dict_[team] = {}
    dict_[team]['FAV#'] = int
    dict_[team]['✅'] = int
    dict_[team]['❌'] = int

In [32]:
dict_ = {}
for team in NBAteams:
    dict_[team] = {}

for team in NBAteams:
    teamDF = NBAdf[NBAdf['TEAM'] == team]
    teamDF = teamDF[teamDF['SPREAD'] < 0]
    dict_[team]['#FAV'] = len(teamDF[teamDF['SPREAD'] < 0])
    dict_[team]['BEAT SPEAD'] = sum(abs(teamDF['SPREAD']) < teamDF['PTSDELTA'])
    dict_[team]['BEAT SPEAD%'] = dict_[team]['BEAT SPEAD'] / dict_[team]['#FAV']
    dict_[team]['LOST SPREAD'] = sum(abs(teamDF['SPREAD']) > teamDF['PTSDELTA'])
    dict_[team]['LOST SPREAD%'] = dict_[team]['LOST SPREAD'] / dict_[team]['#FAV']
    dict_[team]['PUSH'] = sum(abs(teamDF['SPREAD']) ==  teamDF['PTSDELTA'])
    dict_[team]['PUSH%'] = dict_[team]['PUSH'] / dict_[team]['#FAV']
    
# for team in NBAteams:
#     teamDF = NBAdf[NBAdf['TEAM'] == team]
#     teamDF = teamDF[teamDF['SPREAD'] < 0]
#     dict_[team]['#FAV'] = len(teamDF[teamDF['SPREAD'] < 0])
#     dict_[team]['BEAT SPREAD'] = 0
#     dict_[team]['LOST SPREAD'] = 0
#     dict_[team]['PUSH'] = 0
#     for i in range(len(teamDF)):
#         if teamDF.iloc[i]['PTSDELTA'] < 0: #lost
#             dict_[team]['LOST SPREAD'] = dict_[team]['LOST SPREAD'] + 1
#         if teamDF.iloc[i]['PTSDELTA'] > 0: #won
#             if np.abs(teamDF.iloc[i]['SPREAD']) > teamDF.iloc[i]['PTSDELTA']:
#                 dict_[team]['LOST SPREAD'] = dict_[team]['LOST SPREAD'] + 1
#             if np.abs(teamDF.iloc[i]['SPREAD']) == teamDF.iloc[i]['PTSDELTA']:
#                 dict_[team]['PUSH'] = dict_[team]['PUSH'] + 1
#             if np.abs(teamDF.iloc[i]['SPREAD']) < teamDF.iloc[i]['PTSDELTA']:
#                 dict_[team]['BEAT SPREAD'] = dict_[team]['BEAT SPREAD'] + 1
spreadResultsDF = pd.DataFrame(dict_).T

In [33]:
# teamDF = NBAdf[NBAdf['TEAM'] == 'ATL']
# teamDF = teamDF[teamDF['SPREAD'] < 0]
# teamDF.reset_index(drop=True).head()

In [34]:
# abs(teamDF.iloc[2]['SPREAD']) > teamDF.iloc[2]['PTSDELTA']

In [35]:
spreadResultsDF.head()

,#FAV,BEAT SPEAD,BEAT SPEAD%,LOST SPREAD,LOST SPREAD%,PUSH,PUSH%
ATL,43.0,15.0,0.348837,27.0,0.627907,1.0,0.023256
BKN,27.0,14.0,0.518519,12.0,0.444444,1.0,0.037037
BOS,79.0,40.0,0.506329,34.0,0.430380,5.0,0.063291
CHA,6.0,1.0,0.166667,5.0,0.833333,0.0,0.000000
CHI,33.0,15.0,0.454545,17.0,0.515152,1.0,0.030303


In [36]:
dict_ = {}
for team in NBAteams:
    dict_[team] = {}
    dict_[team]['gamesFav+LostSpread'] = pd.DataFrame
    
for team in NBAteams:
    teamDF = NBAdf[NBAdf['TEAM'] == team]
    teamDF = teamDF[teamDF['SPREAD'] < 0]
    teamDF = teamDF.reset_index(drop=True)

    df = pd.DataFrame()
    for i in range(len(teamDF)):
        if teamDF.iloc[i]['PTSDELTA'] < 0:
            df = pd.concat([df,teamDF.iloc[i].to_frame().T],axis=0)
        if teamDF.iloc[i]['PTSDELTA'] > 0:
            if np.abs(teamDF.iloc[i]['SPREAD']) > teamDF.iloc[i]['PTSDELTA']:
                df = pd.concat([df,teamDF.iloc[i].to_frame().T],axis=0)
    dict_[team]['gamesFav+LostSpread'] = df.reset_index(drop=True)

In [37]:
print(NBAteams)

['ATL' 'BKN' 'BOS' 'CHA' 'CHI' 'CLE' 'DAL' 'DEN' 'DET' 'GSW' 'HOU' 'IND'
 'LAC' 'LAL' 'MEM' 'MIA' 'MIL' 'MIN' 'NOP' 'NYK' 'OKC' 'ORL' 'PHI' 'PHX'
 'POR' 'SAC' 'SAS' 'TOR' 'UTA' 'WAS']


In [38]:
def cleanBoxScore(boxScoreDF):
    if 'Unnamed: 22' in boxScoreDF.columns:
            boxScoreDF = boxScoreDF.drop(columns='Unnamed: 22')
    boxScoreDF['PLAYER']
    players = []
    for playerIter in range(5):
        players.append(boxScoreDF['PLAYER'].iloc[playerIter][:-1])
    for playerIter in range(5,len(boxScoreDF)):
        players.append(boxScoreDF['PLAYER'].iloc[playerIter])
    boxScoreDF = boxScoreDF.drop(columns='PLAYER')
    boxScoreDF.insert(0,'PLAYER',players)
    boxScoreDF = boxScoreDF.reset_index(drop=True).dropna()
    
    rowsToDrop = []
    for i in range(len(boxScoreDF)):
        if 'DNP' in boxScoreDF.iloc[i]['MIN'] or ' ' in boxScoreDF.iloc[i]['MIN'] or 'SUSPENSION' in boxScoreDF.iloc[i]['MIN']:
            rowsToDrop.append(i)

    boxScoreDF = boxScoreDF.drop(rowsToDrop)
    boxScoreDF['PTS_'] = boxScoreDF['PTS'].astype(int)
    boxScoreDF = boxScoreDF.drop(columns='PTS')
    boxScoreDF = boxScoreDF.rename(columns={'PTS_':'PTS'})
    return boxScoreDF

In [43]:
results = []
resultsDict = {}
for team in NBAteams:
    resultsDict[team] = []
#players = []

for team in NBAteams:
    print(team)
    dict_[team]['gamesFav+LostSpread'].head()
    teamDF = dict_[team]['gamesFav+LostSpread']
    dateOfFirstGame = NBAdf[NBAdf['TEAM'] == team].head(1)['DATE'].values[0]
    teamDF = teamDF[teamDF['DATE'] > dateOfFirstGame]
    teamDF = teamDF.reset_index(drop=True);
    
    for i in range (len(teamDF)):
        GAMEID = teamDF.iloc[i]['GAMEID']
        GAMEID = str(GAMEID)
        #print(GAMEID,teamDF.iloc[i]['GAME DATE'])
        boxScoreDF = pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv')#.fillna(' ')
        boxScoreDF = boxScoreDF[boxScoreDF['TEAM'] ==  team]
        boxScoreDF.insert(1,'OPPT',teamDF.iloc[i]['OPPT'])
        boxScoreDF = boxScoreDF.reset_index(drop=True).dropna()
        boxScoreDF = cleanBoxScore(boxScoreDF);

        prevGamesDF = NBAdf[NBAdf['TEAM'] == team]
        prevGamesDF  = prevGamesDF[prevGamesDF['DATE'] < teamDF.iloc[i]['DATE']]
        prevGamesDF.reset_index(drop=True)
        
        prevGameIDs = prevGamesDF['GAMEID'].values
        prevOppts = prevGamesDF['OPPT'].values
        
        prevBoxScoresDF = pd.DataFrame()
        for prevIter in range (len(prevGameIDs)):
            GAMEID = str(prevGameIDs[prevIter])
            #print('PREV:',GAMEID)
            prevBoxScoreDF = pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv')#.fillna(' ')
            prevBoxScoreDF = prevBoxScoreDF[prevBoxScoreDF['TEAM'] == team]
            prevBoxScoreDF.insert(1,'OPPT',prevOppts[prevIter])
            prevBoxScoreDF = cleanBoxScore(prevBoxScoreDF)
            prevBoxScoresDF = pd.concat([prevBoxScoresDF,prevBoxScoreDF],axis=0)
        #players.append(prevBoxScoresDF['PLAYER'].values)
        prevBoxScoresDF.groupby('PLAYER')['PTS'].mean()
        boxScoreDF.groupby('PLAYER')['PTS'].mean()
        prevAVGPTS = pd.DataFrame(prevBoxScoresDF.groupby("PLAYER")['PTS'].mean())

        prevAVGPTS = prevAVGPTS.rename(columns={'PTS':'PREV AVG PTS'})
        df = pd.concat([boxScoreDF.groupby("PLAYER")['PTS'].mean(),
               prevAVGPTS.groupby("PLAYER")['PREV AVG PTS'].mean()],axis=1).dropna()
        df['DATE'] = teamDF.iloc[i]['DATE']
        df['TEAM'] = teamDF.iloc[i]['TEAM']
        df['OPPT'] = teamDF.iloc[i]['OPPT']
        df['GAMEID'] = teamDF.iloc[i]['GAMEID']
        df['PTSDELTA'] = teamDF.iloc[i]['PTSDELTA']
        df['SPREAD'] = teamDF.iloc[i]['SPREAD']
        resultsDict[team].append(df)

ATL
BKN
BOS
CHA
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHX
POR
SAC
SAS
TOR
UTA
WAS


In [44]:
#players[0]

In [45]:
#len(players)

In [46]:
# playerSet = set()
# for i in range(len(players)):
#     playerSet.update(set(players[i]))

In [47]:
#len(playerSet)

In [48]:
count = 0
for team in NBAteams:
    FLAG = []
    resultsDF = pd.DataFrame()
    for i in range (len(resultsDict[team])):
        resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
    for i in range(len(resultsDF)):
        if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
            FLAG.append('🚩')
        else: FLAG.append(' ')
    resultsDF['FLAG'] = FLAG
    resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')
    count = count+len(resultsDF[resultsDF['FLAG'] == '🚩'])


In [49]:
print(count)

219


In [50]:
team = 'ATL'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Jalen Johnson,5.0,15.434783,2024-01-13,ATL,WAS,22300543,-28,-8.0,🚩
Saddiq Bey,2.0,12.142857,2023-11-09,ATL,ORL,22300172,1,-3.5,🚩
Trae Young,13.0,27.823529,2024-01-12,ATL,IND,22300533,-18,-6.0,🚩
Trae Young,12.0,26.959184,2024-02-14,ATL,CHA,22300777,-23,-7.0,🚩
Trae Young,11.0,26.660000,2024-02-23,ATL,TOR,22300804,-2,-7.5,🚩
Trae Young,14.0,26.352941,2024-04-10,ATL,CHA,22301159,-1,-8.5,🚩


In [51]:
team = 'BKN'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Cam Thomas,9.0,20.740741,2024-01-07,BKN,POR,22300498,-7,-9.0,🚩
Cam Thomas,8.0,20.600000,2024-01-17,BKN,POR,22300578,-2,-5.0,🚩
Lonnie Walker IV,0.0,14.000000,2024-01-07,BKN,POR,22300498,-7,-9.0,🚩
Lonnie Walker IV,0.0,12.040000,2024-01-27,BKN,HOU,22300645,2,-4.0,🚩


In [52]:
team = 'BOS'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Derrick White,2.0,16.000000,2024-01-27,BOS,LAC,22300646,-19,-7.0,🚩
Derrick White,3.0,15.545455,2024-03-05,BOS,CLE,22300886,-1,-8.5,🚩
Derrick White,4.0,15.826087,2024-02-09,BOS,WAS,22300743,4,-18.0,🚩
Derrick White,2.0,15.380282,2024-04-11,BOS,NYK,22301167,-9,-3.5,🚩
Jaylen Brown,12.0,22.909091,2023-11-19,BOS,MEM,22300213,2,-12.0,🚩
Jaylen Brown,8.0,22.536585,2024-01-27,BOS,LAC,22300646,-19,-7.0,🚩
Jaylen Brown,8.0,22.250000,2024-02-01,BOS,LAL,22300684,-9,-15.5,🚩
Jaylen Brown,11.0,23.166667,2023-11-08,BOS,PHI,22300159,-3,-2.5,🚩
Jayson Tatum,17.0,28.454545,2023-11-17,BOS,TOR,22300031,3,-9.0,🚩


In [53]:
team = 'CHA'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Gordon Hayward,4.0,15.083333,2023-11-22,CHA,WAS,22300225,3,-3.5,🚩
Mark Williams,2.0,13.000000,2023-10-27,CHA,DET,22300077,-12,-4.5,🚩
P.J. Washington,13.0,25.000000,2023-10-27,CHA,DET,22300077,-12,-4.5,🚩


In [54]:
team = 'CLE'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Caris LeVert,6.0,17.500000,2023-11-25,CLE,LAL,22300242,-6,-4.0,🚩
Caris LeVert,5.0,15.533333,2023-11-30,CLE,POR,22300265,-8,-12.5,🚩
Caris LeVert,4.0,15.342105,2024-02-07,CLE,WAS,22300728,8,-11.5,🚩
Darius Garland,6.0,20.300000,2023-11-25,CLE,LAL,22300242,-6,-4.0,🚩
Darius Garland,9.0,20.619048,2024-02-01,CLE,MEM,22300687,7,-8.5,🚩
Darius Garland,8.0,18.218750,2024-02-27,CLE,DAL,22300832,2,-4.0,🚩
Donovan Mitchell,20.0,32.500000,2023-11-08,CLE,OKC,22300167,-8,-2.0,🚩
Donovan Mitchell,10.0,28.545455,2023-11-26,CLE,TOR,22300251,3,-3.5,🚩
Donovan Mitchell,17.0,27.615385,2024-01-07,CLE,SAS,22300497,2,-10.5,🚩


In [55]:
team = 'DAL'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Derrick Jones Jr.,0.0,10.200000,2023-11-05,DAL,CHA,22300143,6,-11.5,🚩
Grant Williams,5.0,17.000000,2023-10-27,DAL,BKN,22300084,5,-6.0,🚩
Kyrie Irving,10.0,24.571429,2023-12-01,DAL,MEM,22300277,-14,-6.0,🚩
Luka Doncic,23.0,33.800000,2023-11-05,DAL,CHA,22300143,6,-11.5,🚩
Luka Doncic,16.0,32.600000,2023-11-14,DAL,NOP,22300022,-21,-3.5,🚩
Luka Doncic,19.0,33.900000,2024-01-01,DAL,UTA,22300457,-37,-4.5,🚩
Luka Doncic,18.0,34.372881,2024-03-19,DAL,SAS,22300994,6,-9.0,🚩
Tim Hardaway Jr.,6.0,17.900000,2024-01-01,DAL,UTA,22300457,-37,-4.5,🚩
Tim Hardaway Jr.,5.0,17.720000,2024-02-12,DAL,WAS,22300768,8,-12.0,🚩


In [56]:
team = 'DEN'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Aaron Gordon,2.0,13.866667,2023-11-24,DEN,HOU,22300048,-19,-3.0,🚩
Aaron Gordon,3.0,13.055556,2023-12-08,DEN,HOU,22301224,-8,-9.0,🚩
Jamal Murray,9.0,19.947368,2023-12-29,DEN,OKC,22300435,-26,-2.0,🚩
Jamal Murray,6.0,21.142857,2024-02-29,DEN,MIA,22300855,6,-6.5,🚩
Kentavious Caldwell-Pope,8.0,20.000000,2023-10-27,DEN,MEM,22300078,4,-6.0,🚩
Kentavious Caldwell-Pope,0.0,10.060606,2024-03-29,DEN,MIN,22301071,-13,-3.0,🚩
Michael Porter Jr.,5.0,16.289474,2024-01-10,DEN,UTA,22300524,-13,-6.5,🚩
Nikola Jokic,18.0,29.666667,2023-11-19,DEN,CLE,22300210,-12,-4.5,🚩
Nikola Jokic,9.0,28.769231,2023-11-20,DEN,DET,22300218,4,-9.5,🚩


In [57]:
team = 'DET'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Bojan Bogdanovic,8.0,21.125000,2023-12-21,DET,UTA,22300374,-8,-2.0,🚩
Jalen Duren,0.0,13.892857,2024-04-01,DET,MEM,22301088,-2,-3.0,🚩


In [58]:
team = 'GSW'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Jonathan Kuminga,0.0,16.222222,2024-04-14,GSW,UTA,22301198,7,-11.0,🚩
Klay Thompson,3.0,17.166667,2023-12-30,GSW,DAL,22300444,-10,-4.0,🚩
Stephen Curry,21.0,33.500000,2023-11-01,GSW,SAC,22300126,1,-6.5,🚩
Stephen Curry,7.0,28.956522,2023-12-17,GSW,POR,22300348,4,-6.0,🚩
Stephen Curry,13.0,27.892857,2023-12-28,GSW,MIA,22300426,-12,-6.5,🚩
Stephen Curry,9.0,27.606061,2024-01-07,GSW,TOR,22300504,-15,-2.5,🚩
Stephen Curry,15.0,27.175439,2024-03-07,GSW,CHI,22300907,-3,-8.5,🚩


In [59]:
team = 'HOU'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Alperen Sengun,10.0,21.934783,2024-01-31,HOU,NOP,22300679,-11,-1.0,🚩
Jabari Smith Jr.,0.0,13.756098,2024-01-31,HOU,NOP,22300679,-11,-1.0,🚩
Jalen Green,5.0,17.111111,2023-12-26,HOU,IND,22300409,-6,-3.0,🚩
Jalen Green,7.0,17.375000,2024-01-20,HOU,UTA,22300598,1,-2.0,🚩
Jalen Green,4.0,18.264151,2024-02-14,HOU,MEM,22300783,-8,-2.5,🚩
Jalen Green,3.0,19.848101,2024-04-11,HOU,UTA,22301168,-3,-11.5,🚩


In [60]:
team = 'IND'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Aaron Nesmith,5.0,16.500000,2023-10-30,IND,CHI,22300102,-7,-3.5,🚩
Aaron Nesmith,0.0,11.181818,2023-11-19,IND,ORL,22300209,-12,-4.5,🚩
Bennedict Mathurin,2.0,13.925926,2023-12-23,IND,ORL,22300391,-7,-2.5,🚩
Bennedict Mathurin,2.0,14.525000,2024-01-19,IND,POR,22300590,-3,-8.5,🚩
Bennedict Mathurin,0.0,14.120000,2024-02-12,IND,CHA,22300761,-9,-10.0,🚩
Myles Turner,6.0,17.590909,2023-12-15,IND,WAS,22300328,-14,-8.0,🚩
Obi Toppin,0.0,11.475000,2024-01-19,IND,POR,22300590,-3,-8.5,🚩
Tyrese Haliburton,12.0,24.700000,2023-11-19,IND,ORL,22300209,-12,-4.5,🚩
Tyrese Haliburton,5.0,22.921053,2024-02-08,IND,GSW,22300733,-22,-6.0,🚩


In [61]:
team = 'LAC'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Ivica Zubac,6.0,20.000000,2023-10-27,LAC,UTA,22300085,-2,-4.0,🚩
James Harden,4.0,17.243902,2024-02-02,LAC,DET,22300688,11,-12.0,🚩
James Harden,6.0,16.984848,2024-03-31,LAC,CHA,22301079,12,-14.0,🚩
James Harden,4.0,16.732394,2024-04-12,LAC,UTA,22301184,-1,-15.5,🚩
Kawhi Leonard,6.0,24.000000,2024-03-12,LAC,MIN,22300944,-18,-5.5,🚩
Norman Powell,2.0,13.510204,2024-02-10,LAC,DET,22300749,6,-17.0,🚩
Paul George,6.0,25.214286,2023-11-27,LAC,DEN,22300257,-9,-11.5,🚩
Paul George,13.0,23.146341,2024-01-29,LAC,CLE,22300658,-10,-1.5,🚩
Paul George,7.0,22.511111,2024-02-07,LAC,NOP,22300731,-11,-5.5,🚩


In [62]:
team = 'LAL'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Anthony Davis,9.0,23.400000,2023-11-15,LAL,SAC,22300198,-15,-1.0,🚩
Anthony Davis,14.0,24.949153,2024-03-06,LAL,SAC,22300901,-10,-3.0,🚩
Anthony Davis,8.0,24.714286,2024-03-16,LAL,GSW,22300973,-7,-1.5,🚩
D'Angelo Russell,4.0,18.384615,2023-11-19,LAL,HOU,22300216,1,-6.5,🚩
D'Angelo Russell,2.0,15.769231,2023-12-20,LAL,CHI,22300370,-16,-3.5,🚩
D'Angelo Russell,6.0,18.032787,2024-03-13,LAL,SAC,22300954,-13,-2.5,🚩
D'Angelo Russell,4.0,18.189189,2024-04-12,LAL,MEM,22301177,3,-16.0,🚩
LeBron James,12.0,25.419355,2024-01-03,LAL,MIA,22300475,-14,-4.5,🚩
Rui Hachimura,0.0,11.607143,2024-01-19,LAL,BKN,22300591,-18,-7.0,🚩


In [63]:
team = 'MEM'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Jaren Jackson Jr.,10.0,21.03125,2024-01-02,MEM,SAS,22300460,8,-11.0,🚩


In [64]:
team = 'MIA'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Bam Adebayo,8.0,21.027778,2024-01-29,MIA,PHX,22300661,-13,-2.5,🚩
Caleb Martin,0.0,10.461538,2024-03-22,MIA,NOP,22301013,-23,-3.0,🚩
Duncan Robinson,0.0,13.500000,2024-01-29,MIA,PHX,22300661,-13,-2.5,🚩
Duncan Robinson,2.0,13.276923,2024-04-04,MIA,PHI,22301111,-4,-2.5,🚩
Jaime Jaquez Jr.,2.0,13.825000,2024-01-29,MIA,PHX,22300661,-13,-2.5,🚩
Jimmy Butler,8.0,21.521739,2023-12-30,MIA,UTA,22300439,-8,-3.0,🚩
Kevin Love,0.0,10.129032,2024-01-19,MIA,ATL,22300587,-1,-7.0,🚩


In [65]:
team = 'MIL'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Bobby Portis,2.0,12.466667,2023-11-24,MIL,WAS,22300049,3,-13.5,🚩
Damian Lillard,6.0,39.000000,2023-10-29,MIL,ATL,22300097,-17,-7.5,🚩
Damian Lillard,12.0,24.285714,2023-11-13,MIL,CHI,22300190,9,-9.5,🚩
Damian Lillard,14.0,25.550000,2023-12-11,MIL,CHI,22300300,4,-11.5,🚩
Damian Lillard,13.0,25.866667,2024-01-01,MIL,IND,22300454,-9,-8.0,🚩
Giannis Antetokounmpo,16.0,27.333333,2023-11-01,MIL,TOR,22300116,-19,-4.5,🚩
Giannis Antetokounmpo,15.0,26.000000,2023-11-08,MIL,DET,22300165,2,-12.0,🚩
Giannis Antetokounmpo,11.0,31.400000,2023-12-19,MIL,SAS,22300361,13,-16.5,🚩
Khris Middleton,5.0,15.215686,2024-04-07,MIL,NYK,22301139,-13,-4.5,🚩


In [66]:
team = 'MIN'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Anthony Edwards,9.0,25.897436,2024-01-22,MIN,CHA,22300610,-3,-14.0,🚩
Anthony Edwards,11.0,26.607143,2024-03-01,MIN,SAC,22300862,-4,-8.5,🚩
Anthony Edwards,13.0,26.344828,2024-03-04,MIN,POR,22300881,5,-15.0,🚩
Anthony Edwards,9.0,26.441176,2024-03-27,MIN,DET,22301056,15,-16.5,🚩
Anthony Edwards,14.0,26.259740,2024-04-12,MIN,ATL,22301178,3,-12.5,🚩
Anthony Edwards,13.0,26.102564,2024-04-14,MIN,PHX,22301194,-19,-2.0,🚩
Jaden McDaniels,0.0,10.857143,2023-11-14,MIN,GSW,22300025,3,-3.5,🚩
Karl-Anthony Towns,10.0,21.964286,2023-12-28,MIN,DAL,22300423,8,-12.0,🚩
Karl-Anthony Towns,9.0,21.551724,2023-12-30,MIN,LAL,22300443,2,-4.5,🚩


In [67]:
team = 'NOP'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Brandon Ingram,8.0,23.636364,2023-12-15,NOP,CHA,22300326,5,-9.5,🚩
CJ McCollum,13.0,23.400000,2023-11-04,NOP,ATL,22300135,-18,-2.0,🚩
CJ McCollum,2.0,18.975610,2024-02-23,NOP,MIA,22300810,-11,-2.0,🚩
CJ McCollum,8.0,18.255319,2024-03-13,NOP,CLE,22300951,-21,-6.0,🚩
Jonas Valanciunas,2.0,12.312500,2024-04-14,NOP,LAL,22301195,-16,-2.0,🚩
Jordan Hawkins,0.0,11.500000,2023-12-15,NOP,CHA,22300326,5,-9.5,🚩
Zion Williamson,12.0,23.028986,2024-04-14,NOP,LAL,22301195,-16,-2.0,🚩


In [68]:
team = 'NYK'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Immanuel Quickley,5.0,16.294118,2023-11-30,NYK,DET,22300268,6,-14.5,🚩
RJ Barrett,9.0,20.500000,2023-11-26,NYK,PHX,22300247,-3,-6.5,🚩
RJ Barrett,9.0,19.588235,2023-12-13,NYK,UTA,22300318,-4,-6.0,🚩


In [69]:
team = 'OKC'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Chet Holmgren,4.0,17.578947,2023-12-06,OKC,HOU,22301212,-9,-1.5,🚩
Chet Holmgren,4.0,16.978261,2024-01-29,OKC,MIN,22300664,-6,-2.5,🚩
Chet Holmgren,7.0,17.177419,2024-03-08,OKC,MIA,22300914,7,-8.5,🚩
Chet Holmgren,2.0,16.943662,2024-03-27,OKC,HOU,22301057,-6,-5.0,🚩
Chet Holmgren,5.0,16.780822,2024-03-31,OKC,NYK,22301082,1,-2.0,🚩
Jalen Williams,7.0,19.333333,2024-03-04,OKC,LAL,22300884,-12,-1.0,🚩
Shai Gilgeous-Alexander,20.0,31.237288,2024-03-04,OKC,LAL,22300884,-12,-1.0,🚩
Shai Gilgeous-Alexander,20.0,31.092308,2024-03-16,OKC,MEM,22300972,6,-10.5,🚩
Shai Gilgeous-Alexander,19.0,30.442857,2024-03-31,OKC,NYK,22301082,1,-2.0,🚩


In [70]:
team = 'ORL'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Cole Anthony,4.0,15.882353,2023-12-01,ORL,WAS,22300274,5,-10.5,🚩
Jalen Suggs,0.0,12.948718,2024-01-26,ORL,MEM,22300638,-1,-6.0,🚩
Paolo Banchero,10.0,21.125000,2023-12-20,ORL,MIA,22300366,-9,-5.5,🚩


In [71]:
team = 'PHI'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Cameron Payne,2.0,13.666667,2024-02-14,PHI,MIA,22300779,-5,-2.5,🚩
Tobias Harris,6.0,18.000000,2023-12-06,PHI,WAS,22301208,5,-10.5,🚩
Tobias Harris,3.0,16.800000,2023-12-18,PHI,CHI,22300351,-4,-10.5,🚩
Tobias Harris,5.0,17.303030,2024-01-05,PHI,NYK,22300483,-36,-6.0,🚩
Tobias Harris,7.0,17.729167,2024-02-22,PHI,NYK,22300794,-14,-1.5,🚩
Tyrese Maxey,16.0,26.205128,2024-01-20,PHI,CHA,22300593,8,-12.0,🚩


In [72]:
team = 'PHX'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Bradley Beal,6.0,16.400000,2023-12-15,PHX,NYK,22300333,-17,-5.0,🚩
Bradley Beal,7.0,18.200000,2024-03-02,PHX,HOU,22300870,-9,-9.0,🚩
Devin Booker,8.0,29.444444,2023-11-29,PHX,TOR,22300260,-7,-3.5,🚩
Devin Booker,16.0,26.566667,2024-01-16,PHX,SAC,22300567,2,-4.5,🚩
Devin Booker,16.0,27.000000,2024-01-22,PHX,CHI,22300612,2,-5.0,🚩
Devin Booker,14.0,27.186441,2024-03-29,PHX,OKC,22301069,-25,-2.5,🚩
Devin Booker,12.0,27.312500,2024-04-09,PHX,LAC,22301156,-13,-9.5,🚩
Eric Gordon,0.0,14.037037,2023-12-31,PHX,ORL,22300450,5,-5.5,🚩
Eric Gordon,3.0,13.250000,2024-01-14,PHX,POR,22300554,11,-14.5,🚩


In [73]:
team = 'POR'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Anfernee Simons,9.0,28.500000,2023-12-14,POR,UTA,22300323,-8,-5.0,🚩
Anfernee Simons,10.0,23.258065,2024-02-25,POR,CHA,22300826,-13,-3.5,🚩
Shaedon Sharpe,4.0,18.772727,2023-12-14,POR,UTA,22300323,-8,-5.0,🚩


In [74]:
team = 'SAC'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
De'Aaron Fox,14.0,31.857143,2023-11-20,SAC,NOP,22300222,-36,-1.5,🚩
De'Aaron Fox,15.0,29.961538,2024-01-03,SAC,ORL,22300476,3,-5.5,🚩
De'Aaron Fox,12.0,27.833333,2024-02-07,SAC,DET,22300732,-13,-13.5,🚩
Domantas Sabonis,10.0,20.833333,2023-11-20,SAC,NOP,22300222,-36,-1.5,🚩
Domantas Sabonis,9.0,19.866667,2023-11-28,SAC,GSW,22300060,1,-2.0,🚩
Harrison Barnes,10.0,33.000000,2023-10-27,SAC,GSW,22300087,-8,-3.0,🚩
Harrison Barnes,3.0,15.600000,2023-11-06,SAC,HOU,22300153,-25,-2.0,🚩
Keegan Murray,5.0,16.208333,2023-12-26,SAC,POR,22300413,-17,-9.0,🚩
Keegan Murray,5.0,16.205128,2024-01-29,SAC,MEM,22300663,9,-11.0,🚩


In [75]:
team = 'TOR'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Dennis Schroder,4.0,16.666667,2023-11-13,TOR,WAS,22300189,4,-8.0,🚩
Gary Trent Jr.,0.0,11.222222,2023-12-08,TOR,CHA,22301216,-3,-5.5,🚩
Scottie Barnes,7.0,20.132075,2024-02-12,TOR,SAS,22300764,-23,-6.0,🚩


In [76]:
team = 'UTA'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Collin Sexton,1.0,12.500000,2023-11-22,UTA,POR,22300237,-16,-3.5,🚩
Collin Sexton,6.0,17.478261,2024-01-29,UTA,BKN,22300660,-33,-1.0,🚩
Lauri Markkanen,13.0,23.833333,2024-01-29,UTA,BKN,22300660,-33,-1.0,🚩


In [77]:
team = 'WAS'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,GAMEID,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,,
Kyle Kuzma,11.0,22.20000,2024-01-20,WAS,SAS,22300594,-4,-2.0,🚩
Kyle Kuzma,12.0,22.38806,2024-03-29,WAS,DET,22301065,-9,-3.5,🚩


In [78]:
################################################
################################################
################################################
################################################
################################################
STOP

NameError: name 'STOP' is not defined

In [ ]:
GAMEID = str(22300104)
boxScoreDF = pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv')#.fillna(' ')
boxScoreDF = boxScoreDF[boxScoreDF['TEAM'] == team]
boxScoreDF.insert(1,'OPPT',teamDF.iloc[i]['OPPT'])
boxScoreDF = boxScoreDF.reset_index(drop=True).dropna()
boxScoreDF = cleanBoxScore(boxScoreDF);
boxScoreDF

In [ ]:
results = []

In [ ]:
for i in range (len(teamDF)):
    GAMEID = teamDF.iloc[i]['GAMEID']
    GAMEID = str(GAMEID)
    print(GAMEID)
    boxScoreDF = pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv')#.fillna(' ')
    boxScoreDF = boxScoreDF[boxScoreDF['TEAM'] == 'ATL']
    boxScoreDF.insert(1,'OPPT',teamDF.iloc[i]['OPPT'])
    boxScoreDF = boxScoreDF.reset_index(drop=True).dropna()
    boxScoreDF = cleanBoxScore(boxScoreDF);
    
    prevGamesDF = NBAdf[NBAdf['TEAM'] == 'ATL']
    prevGamesDF  = prevGamesDF[prevGamesDF['GAME DATE'] < teamDF.iloc[i]['GAME DATE']]
    prevGamesDF.reset_index(drop=True);

    prevGameIDs = prevGamesDF['GAMEID'].values
    prevOppts = prevGamesDF['OPPT'].values

    prevBoxScoresDF = pd.DataFrame()
    for prevIter in range (len(prevGameIDs)):
        GAMEID = str(prevGameIDs[prevIter])
        prevBoxScoreDF = pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv')#.fillna(' ')
        prevBoxScoreDF = prevBoxScoreDF[prevBoxScoreDF['TEAM'] == 'ATL']
        prevBoxScoreDF.insert(1,'OPPT',prevOppts[prevIter])
        prevBoxScoreDF = cleanBoxScore(prevBoxScoreDF)
        prevBoxScoresDF = pd.concat([prevBoxScoresDF,prevBoxScoreDF],axis=0)

    prevBoxScoresDF.groupby("PLAYER")['PTS'].mean()
    boxScoreDF.groupby("PLAYER")['PTS'].mean()
    prevAVGPTS = pd.DataFrame(prevBoxScoresDF.groupby("PLAYER")['PTS'].mean())

    prevAVGPTS = prevAVGPTS.rename(columns={'PTS':'PREV AVG PTS'})
    df = pd.concat([boxScoreDF.groupby("PLAYER")['PTS'].mean(),
           prevAVGPTS.groupby("PLAYER")['PREV AVG PTS'].mean()],axis=1).dropna()
    df['DATE'] = teamDF.iloc[i]['GAME DATE']
    df['TEAM'] = teamDF.iloc[i]['TEAM']
    df['OPPT'] = teamDF.iloc[i]['OPPT']
    results.append(df)
    pd.concat(results)

In [ ]:
pd.concat(results)

In [ ]:
#boxScoreDF

In [ ]:
prevBoxScoreDF.groupby('PLAYER')['PTS'].mean()

In [ ]:
type(prevBoxScoreDF.iloc[0]['PTS'])

In [ ]:
#pd.concat(results)

In [ ]:
prevBoxScoresDF.groupby("PLAYER")['PTS'].mean()

In [ ]:
ATLF.iloc[0].to_frame().T['GAMEID'].values[0]

In [ ]:
GAMEID = str(ATLF.iloc[0].to_frame().T['GAMEID'].values[0])


In [ ]:
pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv').fillna(' ')

In [ ]:
NBAdf.head(1)['GAME DATE'].values

In [ ]:
NBAdf[(NBAdf['TEAM'] == 'TOR') 
               & (NBAdf['OPPT'] == 'SAC') 
               & (NBAdf['GAME DATE'] == datetime.date(2024,3,20))]

In [ ]:
NBAdf[(NBAdf['TEAM'] == 'TOR') 
               & (NBAdf['OPPT'] == 'LAC') 
               & (NBAdf['GAME DATE'] == datetime.date(2024,1,26))]

In [ ]:
GAMEID = NBAdf[(NBAdf['TEAM'] == 'TOR') 
               & (NBAdf['OPPT'] == 'LAC') 
               & (NBAdf['GAME DATE'] == datetime.date(2024,1,26))]['GAMEID'].values[0]
GAMEID = str(GAMEID)
pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+GAMEID +'.csv').fillna(' ')

In [ ]:
#irisDF[~(irisDF['petal_width'] <= 1.75) & (irisDF['petal_length'] <= 4.85)]
GAMEID = NBAdf[(NBAdf['TEAM'] == 'TOR') 
               & (NBAdf['OPPT'] == 'SAC') 
               & (NBAdf['GAME DATE'] == datetime.date(2024,3,20))]['GAMEID'].values[0]
GAMEID = str(GAMEID)
print(GAMEID)

In [ ]:
pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+GAMEID +'.csv').fillna(' ')